In [ ]:
!pip install gpustat

     |████████████████████████████████| 81kB 2.5MB/s 
  Created wheel for gpustat: filename=gpustat-0.6.0-cp36-none-any.whl size=12622 sha256=41659bfaf9c8166b24ae4d3f3b5588d4fba586f1ef38deb4fcd489b90b97b8d1
  Stored in directory: /root/.cache/pip/wheels/48/b4/d5/fb5b7f1d040f2ff20687e3bad6867d63155dbde5a7c10f4293
Successfully built gpustat


In [ ]:
!gpustat -cp

e09b70d4caf0             Sun Jul 12 08:29:04 2020  418.67
[0] Tesla P100-PCIE-16GB | 56'C,   0 % |     0 / 16280 MB |


In [ ]:
import h5py
f = h5py.File("/content/drive/My Drive/CTW2020/Processed Data/udata2020.hdf5","r")
import numpy as np
X = f["H_Est"][:].transpose(0,3,1,2)
f.close()

In [ ]:
X = X.transpose(0,2,3,1)

In [ ]:
X = np.sqrt(X[:,:,:,0]**2 + X[:,:,:,1]**2)

In [ ]:
X.shape

(36192, 56, 924)

In [ ]:
import h5py
f = h5py.File("/content/drive/My Drive/CTW2020/Processed Data/Labelled_1.hdf5","r")
H = f["H_Est"][:]
Pos = f["Pos"][:]
SNR = f["SNR"][:]
f.close()

In [ ]:
import numpy as np
H = np.sqrt(H[:,:,:,0]**2 + H[:,:,:,1]**2)

In [ ]:
H.shape

(4979, 56, 924)

In [ ]:
from sklearn.decomposition import IncrementalPCA

pca = IncrementalPCA(66)
for i in range(56):
  pca.partial_fit(X[:,i,:])
  print(i)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55


In [ ]:
H2 = np.zeros((H.shape[0],H.shape[1],66))
for i in range(56):
  H2[:,i,:] = pca.transform(H[:,i,:])

In [ ]:
print(H2.shape)

(4979, 56, 66)


In [ ]:
from sklearn.model_selection import train_test_split
H_Train, H_Test , Pos_Train , Pos_Test = train_test_split(H2,Pos,test_size=0.05, random_state=42)

In [ ]:
H_Train = H_Train.reshape(-1,56,66,1)

In [ ]:
H_Train.shape

(4730, 56, 66, 1)

In [ ]:
import numpy as np

from tensorflow.keras.models import Model,Sequential , load_model
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Dense , Dropout, Conv2D ,Flatten

from tensorflow.keras.optimizers import Adam, SGD
def classifier():
    model = Sequential()
    model.add(Conv2D(3,(5,5),input_shape=(56,66,1),activation = 'relu' ))
    model.add(Conv2D(6,(5,5),activation = 'relu' ))
    model.add(Conv2D(8,(5,5),activation = 'relu' ))
    model.add(Flatten())
    model.add(Dense(8192 ,activation = 'relu' ))
    model.add(Dense(4096 ,activation = 'relu' ))
    model.add(Dense(3072 ,activation = 'relu' ))
    model.add(Dense(2048 ,activation = 'relu' ))
    model.add(Dense(1024 ,activation = 'relu' ))
    model.add(Dense(512 ,activation = 'relu' ))
    model.add(Dense(128 ,activation = 'relu' ))
    model.add(Dense(32 ,activation = 'relu' ))
    model.add(Dense(8 ,activation = 'relu' ))
    model.add(Dense(3))
    model.compile(loss='mean_squared_error', optimizer = Adam(1e-3)) 
    return model

In [ ]:
model = classifier()
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
earlystopper = EarlyStopping(patience = 80, verbose=1)
checkpointer = ModelCheckpoint('best_PCA3.h5', verbose=1, save_best_only=True)
hist = model.fit(H_Train,Pos_Train , epochs=1000, validation_split=0.1  ,callbacks=[earlystopper, checkpointer],batch_size=4)
val_loss = hist.history['val_loss']

Epoch 1/1000
1065/1065 [==============================] - ETA: 0s - loss: 41779.7344
Epoch 00001: val_loss improved from inf to 24933.27148, saving model to best_PCA3.h5
1065/1065 [==============================] - 56s 53ms/step - loss: 41779.7344 - val_loss: 24933.2715
Epoch 2/1000
1065/1065 [==============================] - ETA: 0s - loss: 25818.7578
Epoch 00002: val_loss did not improve from 24933.27148
1065/1065 [==============================] - 24s 22ms/step - loss: 25818.7578 - val_loss: 25393.3477
Epoch 3/1000
1063/1065 [============================>.] - ETA: 0s - loss: 25700.9395
Epoch 00003: val_loss improved from 24933.27148 to 24351.83789, saving model to best_PCA3.h5
1065/1065 [==============================] - 53s 50ms/step - loss: 25705.9082 - val_loss: 24351.8379
Epoch 4/1000
1063/1065 [============================>.] - ETA: 0s - loss: 25474.1367
Epoch 00004: val_loss improved from 24351.83789 to 23891.80273, saving model to best_PCA3.h5
1065/1065 [====================